In [ ]:
# Citation of the following great resource
# https://www.statology.org/repeated-measures-anova-python/
# https://jbhender.github.io/Stats506/F18/GP/Group16.html
# https://www.statsmodels.org/devel/mixed_linear.html
# Author: Li Zhou, 09202020

In [ ]:
#quick summary:
#For AnovaRM interpretations:
# the pharmacology affected GFP signal, lead to statistically significant differences in dosages (F(3, 9) = 15.5122, p = 0.0007)
# the pharmacology did not significant  affected mcherry signal in dosages (F(3, 9) = 0.4240, p = 0.7405)

#For Linear Mixed models interpretations:
#GFP beta coef: -2.01, 95%CI[-2.95,-1.07]
#mcherry beta coef: 0.08, 95%CI[-1.43, 1.59]

In [1]:
import numpy as np
import pandas as pd
import sys
import statsmodels
import sklearn
import scipy
print('python version:',sys.version)
print("pandas version:",pd.__version__)
print("numpy version:",np.__version__)
print('statsmodels:',statsmodels.__version__)
print('sklearn:',sklearn.__version__)
print('scipy:',scipy.__version__)
# python version: 3.8.3 | packaged by conda-forge | (default, Jun  1 2020, 16:59:10) [MSC v.1916 64 bit (AMD64)]
# pandas version: 1.0.5
# numpy version: 1.18.5
# statsmodels: 0.11.1
# sklearn: 0.23.1
# scipy: 1.3.2

python version: 3.8.3 | packaged by conda-forge | (default, Jun  1 2020, 16:59:10) [MSC v.1916 64 bit (AMD64)]
pandas version: 1.0.5
numpy version: 1.18.5
statsmodels: 0.11.1
sklearn: 0.23.1
scipy: 1.3.2


In [2]:
from sklearn import preprocessing

In [3]:
#import the data
df_GFP = pd.read_csv('emd_GFP.csv')
df_GFP['drug_scaled'] = preprocessing.scale(df_GFP.drug.values)


df_mcherry = pd.read_csv('emd_mcherry.csv')
df_mcherry['drug_scaled'] = preprocessing.scale(df_mcherry.drug.values)
print(df_GFP.head(1))

   patient  drug  response  drug_scaled
0        1     0    4.6135     -1.12833


In [4]:
# Test if the sample differs from a normal distribution / Gaussian.
from scipy import stats
x = df_GFP['response']
k2, p = stats.normaltest(x)
alpha = 0.05
print("p = {:g}".format(p))

if p < alpha:  # null hypothesis: samples comes from a normal distribution /Gaussian. 
    print("The null hypothesis can be rejected")
    print('Probably NOT Gaussian')
else:
    print("The null hypothesis cannot be rejected")
    print('Probably Gaussian or normal distribution')

p = 0.229613
The null hypothesis cannot be rejected
Probably Gaussian or normal distribution


C:\Users\zhouli\Anaconda3\envs\scanpy_sfn-workshop\lib\site-packages\scipy\stats\stats.py:1449: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [5]:
# Test if the sample differs from a normal distribution / Gaussian.
from scipy import stats
x = df_mcherry['response']
k2, p = stats.normaltest(x)
alpha = 0.05
print("p = {:g}".format(p))

if p < alpha:  # null hypothesis: samples comes from a normal distribution /Gaussian. 
    print("The null hypothesis can be rejected")
    print('Probably NOT Gaussian')
else:
    print("The null hypothesis cannot be rejected")
    print('Probably Gaussian or normal distribution')

p = 0.459135
The null hypothesis cannot be rejected
Probably Gaussian or normal distribution


In [6]:
from statsmodels.stats.anova import AnovaRM
#perform the repeated measures ANOVA
print('For GFP:')
print(AnovaRM(data=df_GFP, depvar='response', subject='patient', within=['drug']).fit())
print('_*'*20)
print('_*'*20)
print('\n')
print('For mCherry:')
print(AnovaRM(data=df_mcherry, depvar='response', subject='patient', within=['drug']).fit())

For GFP:
              Anova
     F Value Num DF Den DF Pr > F
---------------------------------
drug 15.5122 3.0000 9.0000 0.0007

_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*
_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*


For mCherry:
              Anova
     F Value Num DF Den DF Pr > F
---------------------------------
drug  0.4240 3.0000 9.0000 0.7405



In [7]:
# Linear Mixed Effects Models in R and Python


In [19]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

#fit the model
mixed = smf.mixedlm("response ~ drug_scaled", df_GFP, groups='patient',re_formula="~drug_scaled")
mixed_fit = mixed.fit()
#print the summary
# print(mixed_fit.summary())
print('the residual probably Gaussian distribution if P is large:',stats.shapiro(mixed_fit.resid)[1])
print('\np value:',mixed_fit.pvalues)
mixed_fit.summary()

the residual probably Gaussian distribution if P is large: 0.667592465877533

p value: Intercept                    0.000029
drug_scaled                  0.000030
patient Var                  0.517679
patient x drug_scaled Cov    0.432228
drug_scaled Var              0.736214
dtype: float64


C:\Users\zhouli\Anaconda3\envs\scanpy_sfn-workshop\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\zhouli\Anaconda3\envs\scanpy_sfn-workshop\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2112: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
C:\Users\zhouli\Anaconda3\envs\scanpy_sfn-workshop\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2170: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
               Mixed Linear Model Regression Results
====================================================================
Model:                  MixedLM     Dependent Variable:     response
No. Observations:       16          Method:                 REML    
No. Groups:             4           Scale:                  0.8031  
Min. group size:        4           Log-Likelihood:         -25.1541
Max. group size:        4           Converged:              Yes     
Mean group size:        4.0                                         
--------------------------------------------------------------------
                          Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------
Intercept                  3.126    0.748  4.179 0.000  1.660  4.593
drug_scaled               -2.010    0.482 -4.175 0.000 -2.954 -1.067
patient Var                2.038    3.516                           
patient x drug_scaled Cov -1.215    1.726                           
drug_scaled Var            0.727    2.407                           
====================================================================

"""

In [ ]:
# beta coef: -2.01, 95%CI[-2.95,-1.07]

In [20]:
#fit the model for mcherry
mixed = smf.mixedlm("response ~ drug_scaled", df_mcherry, groups='patient',re_formula="~drug_scaled")
mixed_fit = mixed.fit()
print('the residual probably Gaussian distribution if P is large:',stats.shapiro(mixed_fit.resid)[1])
#print the summary
print('\np value:',mixed_fit.pvalues)
mixed_fit.summary()

the residual probably Gaussian distribution if P is large: 0.7370639443397522

p value: Intercept                    3.100501e-48
drug_scaled                  9.170127e-01
patient Var                           NaN
patient x drug_scaled Cov    2.323287e-01
drug_scaled Var                       NaN
dtype: float64


C:\Users\zhouli\Anaconda3\envs\scanpy_sfn-workshop\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\zhouli\Anaconda3\envs\scanpy_sfn-workshop\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2112: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
C:\Users\zhouli\Anaconda3\envs\scanpy_sfn-workshop\lib\site-packages\statsmodels\base\model.py:1354: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
C:\Users\zhouli\Anaconda3\envs\scanpy_sfn-workshop\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\zhouli\Anaconda3\envs\scanpy_sfn-workshop\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  

<class 'statsmodels.iolib.summary2.Summary'>
"""
               Mixed Linear Model Regression Results
===================================================================
Model:                 MixedLM     Dependent Variable:     response
No. Observations:      16          Method:                 REML    
No. Groups:            4           Scale:                  2.5467  
Min. group size:       4           Log-Likelihood:         -31.6836
Max. group size:       4           Converged:              Yes     
Mean group size:       4.0                                         
-------------------------------------------------------------------
                          Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                 9.338    0.640 14.593 0.000  8.084 10.592
drug_scaled               0.080    0.770  0.104 0.917 -1.430  1.590
patient Var               1.001                                    
patient x drug_scaled Cov 1.319    0.692                           
drug_scaled Var           1.737                                    
===================================================================

"""